# Per-Image Metrics

How to use, plot, and compare models using per-image [pixel-wise] metrics. 

# Installing Anomalib

The easiest way to install anomalib is to use pip. You can install it from the command line using the following command:


In [ ]:
%pip install anomalib

In [ ]:
%load_ext autoreload

In [ ]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Data

We will use MVTec AD DataModule. 

> See [these notebooks](https://github.com/openvinotoolkit/anomalib/tree/main/notebooks/100_datamodules) for more details on datamodules. 

We assume that `datasets` directory is created in the `anomalib` root directory and `MVTec` dataset is located in `datasets` directory.

In [ ]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded to this directory.
dataset_root = Path.cwd().parent.parent / "datasets" / "MVTec"

We will be working on a segmentation task. 

In [ ]:
from anomalib.data import TaskType

task = TaskType.SEGMENTATION

And with the `hazelnut` category at resolution of 256x256 pixels.

In [ ]:
from anomalib.data.mvtec import MVTec

datamodule = MVTec(
    root=dataset_root,
    category="hazelnut",
    image_size=256,
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=8,
    task=task,
)
datamodule.setup()
i, data = next(enumerate(datamodule.test_dataloader()))
print(f'Image Shape: {data["image"].shape} Mask Shape: {data["mask"].shape}')

# Model

We will use PaDiM.

> See [these notebooks](https://github.com/openvinotoolkit/anomalib/tree/main/notebooks/200_models) for more details on models. 

The next cell instantiates and trains the model.

The `MetricsConfigurationCallback()` will not have metric because they will be created manually.

In [ ]:
from pytorch_lightning import Trainer

from anomalib.utils.callbacks import MetricsConfigurationCallback, PostProcessingConfigurationCallback
from anomalib.post_processing import NormalizationMethod, ThresholdMethod
from anomalib.models import Padim

model = Padim(
    input_size=(256, 256),
    layers=[
        "layer1",
        "layer2",
    ],
    backbone="resnet18",
    pre_trained=True,
)

trainer = Trainer(
    callbacks=[
        PostProcessingConfigurationCallback(
            normalization_method=NormalizationMethod.MIN_MAX,
            threshold_method=ThresholdMethod.ADAPTIVE,
        ),
        MetricsConfigurationCallback(),
    ],
    max_epochs=1,
    num_sanity_val_steps=0,  # does not work for padim
    accelerator="auto",
)

trainer.fit(datamodule=datamodule, model=model)

# Process test images

This part is usually happening automatically but here we want to extract the outputs manually.

In [ ]:
import torch

model.eval()

outputs = []
for batchidx, batch in enumerate(datamodule.test_dataloader()):
    outputs.append(model.test_step_end(model.test_step(batch, batchidx)))

anomaly_maps = torch.squeeze(torch.cat([o["anomaly_maps"] for o in outputs], dim=0))
masks = torch.squeeze(torch.cat([o["mask"] for o in outputs], dim=0)).int()
print(f"{anomaly_maps.shape=} {masks.shape=}")

# Pixel-wise [set] metrics

The usual set pixel-wise metrics. 

Only one value for the whole test set is measured.

In [ ]:
import torch
from anomalib.utils.metrics import AUROC, AUPR

metrics = [AUROC(), AUPR()]

for metric in metrics:
    metric.cpu()
    metric.update(anomaly_maps, masks)

for metric in metrics:
    print(f"{metric}={metric.compute()}")
    metric.generate_figure()

# `AUPImO` (init, update, compute)

Area Under the Per-Image Overlap (`AUPImO`) 

Let's instantiate, load the data, then compute PImO curves and their AUCs (AUPImO scores).

In [ ]:
%%time
%autoreload 2

from anomalib.utils.metrics.perimg import AUPImO

aupimo = AUPImO()
aupimo.cpu()
aupimo.update(anomaly_maps, masks)

pimoresult, aucs = aupimo.compute()
(thresholds, fprs, shared_fpr, tprs, image_classes) = pimoresult

In [ ]:
pimoresult?

In [ ]:
print(f"{thresholds.shape=}")
print(f"{fprs.shape=}")
print(f"{shared_fpr.shape=}")
print(f"{tprs.shape=}")
print(f"{image_classes.shape=}")
print(f"{aucs.shape=}")

# `PImO` curves (plot)

The PImO curve has a shared X-axis and a per-image Y-axis.

The X-axis:
- is a metric of False Positives only in the normal images (here it is the set-FPR)
- is shared by all image instances

The Y-axis: 
- is the **overlap** between the binary predicted mask and the ground truth mask, which corresponds to the True Positive Rate (TPR) in a single image
- has one value per image, so there is one PImO curve per image. 

In [ ]:
aupimo.plot_all_pimo_curves()
# TODO add functional interface

# `AUPImO` = AUC(`PImO`)

The Area Under the Curve (AUC) is, by consequence, computed for each image, which will be used as is score.

Notice that `aucs` has the number of images seen in `outputs` (cf. `masks` below).

`aucs` has `nan` values for the normal images because the `Per-Image Overlap`, by definition, is not defined on them (they do not have any positive/anomalous pixels).

This is done by design choice so the indexes in `aucs` correspond to the indices of the actual images.

In [ ]:
print(f"{masks.shape[0]=}  ==  {aucs.shape[0]=}")
print(aucs)

# `AUPImO` distribuion (boxplot)

One can now analyze the distribution of this True Posivity metric across images and take statistics from the test set (e.g. with `sp.stats.describe`).

`AUPImO` has an integrated feature to plot a boxplot from the distribution and inspect representative cases using its statistics.

In [ ]:
import scipy as sp

print(sp.stats.describe(aucs[~torch.isnan(aucs)]))  # `~torch.isnan(aucs)` is removing the `nan`s
aupimo.plot_boxplot()
aupimo.boxplot_stats()[-3:]
# TODO add functional interface

# Representative samples (curve + boxplot)

The two plots (`PImO` curve + `AUPImO` boxplot) are combined with the method `AUPImO.plot()`.

The `PImO` curves are plot only for the samples that correspond to the boxplot's statistics (see `AUPImO.boxplot_stats()`).

In [ ]:
aupimo.plot()

# Appendix

# scrathc/cache (please ignore this section)

In [ ]:
del AUPImO

In [ ]:
from pathlib import Path

(CACHE := Path.home() / ".cache").mkdir(exist_ok=True)

In [ ]:
torch.save(anomaly_maps, CACHE / "anomaly_maps.pt")
torch.save(masks, CACHE / "masks.pt")

In [ ]:
from pathlib import Path

(CACHE := Path.home() / ".cache").mkdir(exist_ok=True)
import torch

anomaly_maps = torch.load(CACHE / "anomaly_maps.pt")
masks = torch.load(CACHE / "masks.pt")

In [ ]:
%load_ext autoreload